#### DiC Assignment 2

Group 6
Members:
 Theresa Mayer
 Theresa Brucker
 Jan Tölken
 Can Kenan Kandil 
 Thomas Klar


## Part 1

## Part 2

In [1]:
import os
import sys

#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF, ChiSqSelector, IndexToString, StringIndexer, CountVectorizer, Normalizer
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Assignment2").getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.


25/05/11 09:20:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
path = "hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
input_file = spark.read.format("json").load(path).select("category", "reviewText")

In [5]:
input_file.show(n=5)

+--------------------+--------------------+
|            category|          reviewText|
+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...|
|Patio_Lawn_and_Garde|This is a very ni...|
|Patio_Lawn_and_Garde|The metal base wi...|
|Patio_Lawn_and_Garde|For the most part...|
|Patio_Lawn_and_Garde|This hose is supp...|
+--------------------+--------------------+
only showing top 5 rows



### Label Encoding

In [6]:
indexer = StringIndexer(inputCol="category", outputCol="label")
indexModel = indexer.fit(input_file)
input_file_1 = indexModel.transform(input_file)

In [7]:
reindexer = IndexToString(inputCol=indexer.getOutputCol(), outputCol="category_reindexed")
reindexer.transform(input_file_1).show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|  category_reindexed|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|For the most part...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|Patio_Lawn_and_Garde|
+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



### Tokenization

In [8]:
tokenizer = RegexTokenizer(inputCol='reviewText', outputCol='tokens', pattern=r"[ \t\d(){}\[\].!?;:,\-=\"~#@&*%€$§\\'\n\r\/]+", minTokenLength=2, toLowercase=True)

In [9]:
input_2 = tokenizer.transform(input_file_1)
input_2.show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|              tokens|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|[this, was, gift,...|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|[this, is, very, ...|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|[the, metal, base...|
|Patio_Lawn_and_Garde|For the most part...| 18.0|[for, the, most, ...|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|[this, hose, is, ...|
+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



In [10]:
import os
print(os.getcwd())

/home/e01609010/Exercise_2


### Stopword Removal

In [11]:
stopword_file = "stopwords.txt"
with open(stopword_file, 'r', encoding='utf-8') as f:
    # Strip whitespace and convert to lowercase
    stopwords = [line.strip() for line in f]

In [12]:
stopword_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                                    outputCol="tokens_nostop",
                                    stopWords=stopwords)

In [13]:
input_3 = stopword_remover.transform(input_2)
input_3.select("tokens", "tokens_nostop").show(n=5)

+--------------------+--------------------+
|              tokens|       tokens_nostop|
+--------------------+--------------------+
|[this, was, gift,...|[gift, husband, m...|
|[this, is, very, ...|[nice, spreader, ...|
|[the, metal, base...|[metal, base, hos...|
|[for, the, most, ...|[part, works, pre...|
|[this, hose, is, ...|[hose, supposed, ...|
+--------------------+--------------------+
only showing top 5 rows



### TF-IDF Calculation

In [14]:
tf = CountVectorizer(inputCol=stopword_remover.getOutputCol(), 
                      outputCol="tf_output", 
                      vocabSize=40_000)

In [15]:
idf = IDF(inputCol=tf.getOutputCol(), 
          outputCol="tfidf_output",
          minDocFreq=4)

In [16]:
tfmodel = tf.fit(input_3)
input_4 = tfmodel.transform(input_3)
input_4.select("tokens_nostop", "tf_output").show(n=5)

+--------------------+--------------------+
|       tokens_nostop|           tf_output|
+--------------------+--------------------+
|[gift, husband, m...|(40000,[2,3,7,8,3...|
|[nice, spreader, ...|(40000,[0,1,3,21,...|
|[metal, base, hos...|(40000,[4,10,29,1...|
|[part, works, pre...|(40000,[1,3,4,9,1...|
|[hose, supposed, ...|(40000,[12,32,42,...|
+--------------------+--------------------+
only showing top 5 rows



In [17]:
idfModel = idf.fit(input_4)
input_5 = idfModel.transform(input_4)
input_5.select("tf_output", "tfidf_output").show(n=5)

+--------------------+--------------------+
|           tf_output|        tfidf_output|
+--------------------+--------------------+
|(40000,[2,3,7,8,3...|(40000,[2,3,7,8,3...|
|(40000,[0,1,3,21,...|(40000,[0,1,3,21,...|
|(40000,[4,10,29,1...|(40000,[4,10,29,1...|
|(40000,[1,3,4,9,1...|(40000,[1,3,4,9,1...|
|(40000,[12,32,42,...|(40000,[12,32,42,...|
+--------------------+--------------------+
only showing top 5 rows



### Selection of top 2000 features

In [18]:
chisq = ChiSqSelector(featuresCol=idf.getOutputCol(),
                      labelCol="label",
                      outputCol="features",
                      numTopFeatures=2000)

In [19]:
chisqModel = chisq.fit(input_5)
input_6 = chisqModel.transform(input_5)
input_6.select("features").show(n=5)

+--------------------+
|            features|
+--------------------+
|(2000,[2,3,7,8,35...|
|(2000,[0,1,3,21,3...|
|(2000,[4,10,174,3...|
|(2000,[1,3,4,9,10...|
|(2000,[12,29,101,...|
+--------------------+
only showing top 5 rows



### Pipeline Creation

In [20]:
def get_pipeline(n_features=2000):
    chisq.setNumTopFeatures(n_features)
    pipeline = Pipeline(stages=[
        indexer,
        tokenizer,
        stopword_remover,
        tf,
        idf,
        chisq
    ])
    return pipeline

In [21]:
pipeline = get_pipeline(n_features=2000)
preprocessing_pipeline = pipeline.fit(input_file)
preprocessing_pipeline.transform(input_file).select("label", "features").show(n=5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 18.0|(2000,[2,3,7,8,35...|
| 18.0|(2000,[0,1,3,21,3...|
| 18.0|(2000,[4,10,174,3...|
| 18.0|(2000,[1,3,4,9,10...|
| 18.0|(2000,[12,29,101,...|
+-----+--------------------+
only showing top 5 rows



### Export most important tokens to file

In [22]:
def get_top_terms_from_pipeline(pipeline):
    n = len(pipeline.stages[5].selectedFeatures)


    vocab = pipeline.stages[3].vocabulary.copy()
    top_words = " ".join(sorted([vocab[i] for i in pipeline.stages[5].selectedFeatures]))
    
    with open("output_ds.txt", "w") as f:
        f.write(top_words)
        
    return n

In [23]:
get_top_terms_from_pipeline(preprocessing_pipeline)

2000

# Part 3

Import the necessary libraries from pyspark

In [24]:
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import time
#to ignore the warning messages 
spark.sparkContext.setLogLevel("ERROR")

Now we split the data into training, validation and testing 

In [25]:
train_data, val_data, test_data = input_file.randomSplit([0.6,0.2,0.2], seed = 1234)

We want to get additional steps into the pipeline, so for that we add a normalizer, a svm and ovr 

In [ ]:
#Normalizer 
normalizer = Normalizer(inputCol=selector.getOutputCol(), outputCol="norm_features")

#SVR: we will not add features, as we will compute the best option later on in CV
svm = LinearSVC(featuresCol = "norm_features", output = "norm_features")

#OVR 
ovr = OneVsRest(classifier = svm, labelCol = "label",featuresCol = "norm_features")

Now we preprocess the data with the exisiting pipeline 

In [ ]:
#pipeline = get_pipeline(n_features=2000)
pre_model = pipeline.fit(train_data)
# Cache the precomputed results
preprocessed_train = pre_model.transform(train_data).cache()
print('train done')
preprocessed_test = pre_model.transform(test_data).cache()
print('test done')
preprocessed_val= pre_model.transform(val_data).cache()
print('val done')

## Now we do cross validation

In [ ]:
#first we create parameters for best model selection
best_model = None
best_f1_score = float('-inf')
best_params = None

We create the cross validation grid to compare the specific parameters with each other 

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(LinearSVC.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(LinearSVC.maxIter, [10, 50]) \
    .addGrid(LinearSVC.standardization, [True, False]) \
    .build()

Now we will iterate over all parameters in the grid 

In [ ]:
# Iterate over the parameter grid
for params in paramGrid:
    regParam = params[LinearSVC.regParam]
    maxIter = params[LinearSVC.maxIter]
    standardization = params[LinearSVC.standardization]

    print(f'fregParam={regParam}, maxIter={maxIter}, standardization={standardization}')

    normalizer = Normalizer(inputCol="features", outputCol="norm_features")
    svm = LinearSVC(featuresCol="norm_features", labelCol="label", regParam=regParam, maxIter=maxIter, standardization=standardization)
    ovr = OneVsRest(classifier=svm, labelCol="label")

    pipeline = Pipeline(stages=[normalizer, ovr])

    # Create a new pipeline model with the current parameter combination
    start_time = time.time()
    pipeline_model = pipeline.fit(preprocessed_train)
    fit_time = time.time() - start_time
    print(f'fit_time={fit_time}')

    # Make predictions on the validation set
    predictions = pipeline_model.transform(preprocessed_val)
    transform_time = time.time() - start_time - fit_time
    print(f'transform_time={transform_time}')

    # Evaluate the model
    f1_score = evaluator.evaluate(predictions)
    evaluate_time = time.time() - start_time - fit_time - transform_time
    print(f'evaluate_time={evaluate_time}')

    print(f'f1_score={f1_score}')

    if f1_score > best_f1_score:
        best_f1_score = f1_score
        best_model = pipeline_model
        best_params = f'regParam={regParam}, maxIter={maxIter}, standardization={standardization}'



### Evaluation 

In [34]:

spark.sparkContext.setLogLevel("ERROR")

# Create parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(LinearSVC.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(LinearSVC.maxIter, [10, 50]) \
    .addGrid(LinearSVC.standardization, [True, False]) \
    .build()

train_df = preprocessed_train_df_2000
validation_df = preprocessed_validation_df_2000
test_df = preprocessed_test_df_2000


# Iterate over the parameter grid
for params in paramGrid:
    regParam = params[LinearSVC.regParam]
    maxIter = params[LinearSVC.maxIter]
    standardization = params[LinearSVC.standardization]

    print(f'fregParam={regParam}, maxIter={maxIter}, standardization={standardization}')

    normalizer = Normalizer(inputCol="features", outputCol="norm_features_2000")
    svm = LinearSVC(featuresCol="norm_features_2000", labelCol="label", regParam=regParam, maxIter=maxIter, standardization=standardization)
    ovr = OneVsRest(classifier=svm, labelCol="label")

    pipeline = Pipeline(stages=[normalizer, ovr])

    # Create a new pipeline model with the current parameter combination
    start_time = time.time()
    pipeline_model = pipeline.fit(train_df)
    fit_time = time.time() - start_time
    print(f'fit_time={fit_time}')

    # Make predictions on the validation set
    predictions = pipeline_model.transform(validation_df)
    transform_time = time.time() - start_time - fit_time
    print(f'transform_time={transform_time}')

    # Evaluate the model
    f1_score = evaluator.evaluate(predictions)
    evaluate_time = time.time() - start_time - fit_time - transform_time
    print(f'evaluate_time={evaluate_time}')

    print(f'f1_score={f1_score}')

    if f1_score > best_f1_score:
        best_f1_score = f1_score
        best_model = pipeline_model
        best_params = f'regParam={regParam}, maxIter={maxIter}, standardization={standardization}'



fregParam=0.01, maxIter=10, standardization=True
fit_time=110.07819104194641
transform_time=9.401306867599487
evaluate_time=26.320063591003418
f1_score=0.5963699016803471
fregParam=0.01, maxIter=10, standardization=False
fit_time=92.75269603729248
transform_time=8.642877340316772
evaluate_time=18.99094271659851
f1_score=0.597002399556832
fregParam=0.01, maxIter=50, standardization=True
fit_time=326.4193527698517
transform_time=8.712846755981445
evaluate_time=19.49401593208313
f1_score=0.5929666346012658
fregParam=0.01, maxIter=50, standardization=False
fit_time=328.99077558517456
transform_time=8.457764625549316
evaluate_time=18.910300731658936
f1_score=0.5939834684873975
fregParam=0.1, maxIter=10, standardization=True
fit_time=88.93471503257751
transform_time=8.575965166091919
evaluate_time=18.862366914749146
f1_score=0.580914042899032
fregParam=0.1, maxIter=10, standardization=False
fit_time=91.14981198310852
transform_time=8.866122484207153
evaluate_time=18.89891266822815
f1_score=0

In [35]:
print(f"Best F1 Score: {best_f1_score}")
print(f"Best Parameters: {best_params}")

Best F1 Score: 0.597002399556832
Best Parameters: regParam=0.01, maxIter=10, standardization=False


In [36]:
# Predictions on the test data
try:
    print("Making predictions...")
    predictions = best_model.transform(test_df)
    print("Predictions made.")
except Exception as e:
    print(f"Error during predictions: {e}")

Making predictions...
Predictions made.


In [37]:
# Evaluating the model
try:
    print("Evaluating the model...")
    f1_score = evaluator.evaluate(predictions)
    print(f"F1 Score: {f1_score}")
except Exception as e:
    print(f"Error during evaluation: {e}")

Evaluating the model...
F1 Score: 0.5978292729528246


In [36]:
train_data = train_data.cache()
cvModel = cv.fit(train_data)

25/05/10 21:16:46 WARN CacheManager: Asked to cache already cached data.
25/05/10 21:16:46 WARN CacheManager: Asked to cache already cached data.


25/05/10 21:16:47 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:16:47 WARN DAGScheduler: Broadcasting large task binary with size 1112.4 KiB


25/05/10 21:16:51 WARN DAGScheduler: Broadcasting large task binary with size 1114.4 KiB


25/05/10 21:16:57 WARN DAGScheduler: Broadcasting large task binary with size 1114.4 KiB


25/05/10 21:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB


...

25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:00 WARN DAGScheduler: Broadcasting large task binary with size 1129.7 KiB


25/05/10 21:49:04 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:04 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:04 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:05 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB


25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB
25/05/10 21:49:09 WARN DAGScheduler: Broadcasting large task binary with size 1130.9 KiB
25/05/10 21:49:10 WARN DAGScheduler: Broadcasting large task binary with size 1130.8 KiB


[Stage 8906:> (0 + 2) / 2][Stage 8907:> (0 + 0) / 2][Stage 8908:> (0 + 0) / 2]

KeyboardInterrupt: 

In [ ]:
val_predictions = cvModel.transform(val_data)
val_f1 = evaluator.evaluate(val_predictions)
print(f"F1-Score: {val_f1}")